In [ ]:
"""
Enhanced ECG Machine Learning Classification with Statistical Analysis and Interpretability
Addresses: Statistical Significance, Computational Cost, Interpretability
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import psutil
import os
from datetime import datetime

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (classification_report, confusion_matrix, roc_curve,
                            roc_auc_score, precision_recall_curve, average_precision_score,
                            matthews_corrcoef, cohen_kappa_score)
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

# For statistical testing
from scipy import stats
from scipy.stats import ttest_rel, wilcoxon
import statsmodels.stats.api as sms

# For interpretability
import shap
try:
    import lime
    import lime.lime_tabular
    LIME_AVAILABLE = True
except ImportError:
    LIME_AVAILABLE = False
    print("LIME not available. Install with: pip install lime")

import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('ggplot')
sns.set(style="whitegrid")

# ==========================================
# UTILITY FUNCTIONS FOR COMPUTATIONAL COST
# ==========================================

def get_memory_usage():
    """Get current memory usage in MB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024  # Convert to MB

def track_computational_cost(func):
    """Decorator to track time and memory usage"""
    def wrapper(*args, **kwargs):
        start_time = time.time()
        start_memory = get_memory_usage()

        result = func(*args, **kwargs)

        end_time = time.time()
        end_memory = get_memory_usage()

        cost_info = {
            'time_seconds': end_time - start_time,
            'memory_mb': end_memory - start_memory,
            'peak_memory_mb': end_memory
        }

        return result, cost_info
    return wrapper

# ==========================================
# DATA LOADING AND PREPROCESSING
# ==========================================

print("="*80)
print("ENHANCED ECG CLASSIFICATION WITH STATISTICAL ANALYSIS AND INTERPRETABILITY")
print("="*80)

# Dataset paths - UPDATE THESE TO YOUR ACTUAL PATHS
MITBIH_TRAIN_PATH = "/content/drive/MyDrive/ECG_Datasets/ECG_ML/mitbih_test.csv"
MITBIH_TEST_PATH = "/content/drive/MyDrive/ECG_Datasets/ECG_ML/mitbih_test.csv"
PTBDB_NORMAL_PATH = "/content/drive/MyDrive/ECG_Datasets/ECG_ML/ptbdb_normal.csv"
PTBDB_ABNORMAL_PATH = "/content/drive/MyDrive/ECG_Datasets/ECG_ML/ptbdb_abnormal.csv"

# Results directory
RESULTS_DIR = "enhanced_results"
os.makedirs(RESULTS_DIR, exist_ok=True)

print("\n1. LOADING DATASETS...")
print("-" * 80)

# Load datasets with error handling
try:
    mitbih_train = pd.read_csv(MITBIH_TRAIN_PATH, header=None)
    mitbih_test = pd.read_csv(MITBIH_TEST_PATH, header=None)
    ptbdb_normal = pd.read_csv(PTBDB_NORMAL_PATH, header=None)
    ptbdb_abnormal = pd.read_csv(PTBDB_ABNORMAL_PATH, header=None)

    print(f"✓ MIT-BIH train: {mitbih_train.shape}")
    print(f"✓ MIT-BIH test: {mitbih_test.shape}")
    print(f"✓ PTB-DB normal: {ptbdb_normal.shape}")
    print(f"✓ PTB-DB abnormal: {ptbdb_abnormal.shape}")
except FileNotFoundError as e:
    print(f"ERROR: {e}")
    print("Please update the file paths in the script.")
    exit(1)

# Document dataset characteristics
print("\n2. DATASET CHARACTERISTICS")
print("-" * 80)
dataset_info = {
    'MIT-BIH': {
        'Source': 'MIT-BIH Arrhythmia Database',
        'Sampling Rate': '125 Hz (resampled from 360 Hz)',
        'Signal Length': '187 samples per heartbeat',
        'Number of Leads': '1 lead (modified lead II)',
        'Total Samples Train': len(mitbih_train),
        'Total Samples Test': len(mitbih_test)
    },
    'PTB-DB': {
        'Source': 'PTB Diagnostic ECG Database',
        'Sampling Rate': '125 Hz',
        'Signal Length': '187 samples per heartbeat',
        'Number of Leads': '1 lead (preprocessed)',
        'Normal Samples': len(ptbdb_normal),
        'Abnormal Samples': len(ptbdb_abnormal)
    }
}

for dataset, info in dataset_info.items():
    print(f"\n{dataset}:")
    for key, value in info.items():
        print(f"  {key}: {value}")

# Save dataset documentation
with open(f"{RESULTS_DIR}/dataset_characteristics.txt", 'w') as f:
    f.write("DATASET CHARACTERISTICS FOR REVIEW\n")
    f.write("="*80 + "\n\n")
    for dataset, info in dataset_info.items():
        f.write(f"{dataset}:\n")
        for key, value in info.items():
            f.write(f"  {key}: {value}\n")
        f.write("\n")

# ==========================================
# BINARY CLASSIFICATION PREPARATION
# ==========================================

print("\n3. PREPARING BINARY CLASSIFICATION")
print("-" * 80)

def mitbih_to_binary(label):
    """Convert MIT-BIH multi-class to binary: 0=Normal, 1=Abnormal"""
    return 0 if label == 0 else 1

# MIT-BIH processing
X_mitbih_train = mitbih_train.iloc[:, :-1]
y_mitbih_train_binary = mitbih_train.iloc[:, -1].apply(mitbih_to_binary)

X_mitbih_test = mitbih_test.iloc[:, :-1]
y_mitbih_test_binary = mitbih_test.iloc[:, -1].apply(mitbih_to_binary)

# PTB-DB processing
ptbdb_normal['label'] = 0
ptbdb_abnormal['label'] = 1
ptbdb_combined = pd.concat([ptbdb_normal, ptbdb_abnormal], axis=0)
X_ptbdb = ptbdb_combined.iloc[:, :-1]
y_ptbdb = ptbdb_combined.iloc[:, -1]

print(f"MIT-BIH train - Normal: {(y_mitbih_train_binary==0).sum()}, Abnormal: {(y_mitbih_train_binary==1).sum()}")
print(f"PTB-DB - Normal: {(y_ptbdb==0).sum()}, Abnormal: {(y_ptbdb==1).sum()}")

# ==========================================
# FEATURE ENGINEERING
# ==========================================

print("\n4. FEATURE EXTRACTION")
print("-" * 80)

def extract_time_features(signals_df):
    """Extract comprehensive time-domain ECG features"""
    features = pd.DataFrame()

    # Basic statistics
    features['mean'] = signals_df.mean(axis=1)
    features['std'] = signals_df.std(axis=1)
    features['min'] = signals_df.min(axis=1)
    features['max'] = signals_df.max(axis=1)
    features['range'] = features['max'] - features['min']
    features['median'] = signals_df.median(axis=1)

    # RMS (Root Mean Square)
    features['rms'] = np.sqrt((signals_df**2).mean(axis=1))

    # Skewness and Kurtosis
    features['skewness'] = signals_df.skew(axis=1)
    features['kurtosis'] = signals_df.kurtosis(axis=1)

    # Quartiles
    features['q25'] = signals_df.quantile(0.25, axis=1)
    features['q75'] = signals_df.quantile(0.75, axis=1)
    features['iqr'] = features['q75'] - features['q25']

    # First derivative statistics
    diff1 = signals_df.diff(axis=1).iloc[:, 1:]
    features['mean_d1'] = diff1.mean(axis=1)
    features['std_d1'] = diff1.std(axis=1)
    features['max_d1'] = diff1.max(axis=1)
    features['min_d1'] = diff1.min(axis=1)

    # Second derivative statistics
    diff2 = signals_df.diff(axis=1).diff(axis=1).iloc[:, 2:]
    features['mean_d2'] = diff2.mean(axis=1)
    features['std_d2'] = diff2.std(axis=1)
    features['max_d2'] = diff2.max(axis=1)

    return features

# Extract features
print("Extracting features from MIT-BIH...")
X_mitbih_all = pd.concat([X_mitbih_train, X_mitbih_test], axis=0)
y_mitbih_all = pd.concat([y_mitbih_train_binary, y_mitbih_test_binary], axis=0)
X_mitbih_features = extract_time_features(X_mitbih_all)

print("Extracting features from PTB-DB...")
X_ptbdb_features = extract_time_features(X_ptbdb)

# Combine datasets
X_combined_features = pd.concat([X_mitbih_features, X_ptbdb_features], axis=0)
y_combined = pd.concat([y_mitbih_all, y_ptbdb], axis=0)

print(f"Total features extracted: {X_combined_features.shape[1]}")
print(f"Total samples: {X_combined_features.shape[0]}")
print(f"Class distribution - Normal: {(y_combined==0).sum()}, Abnormal: {(y_combined==1).sum()}")

# ==========================================
# TRAIN-TEST SPLIT
# ==========================================

print("\n5. TRAIN-TEST SPLIT")
print("-" * 80)
print("NOTE: Using stratified random split. Patient-level splitting requires patient IDs.")
print("Recommendation: If patient IDs available, use GroupShuffleSplit to prevent data leakage.")

X_train_features, X_test_features, y_train, y_test = train_test_split(
    X_combined_features, y_combined,
    test_size=0.2,
    random_state=42,
    stratify=y_combined
)

print(f"Training set: {X_train_features.shape[0]} samples")
print(f"Test set: {X_test_features.shape[0]} samples")
print(f"Train - Normal: {(y_train==0).sum()}, Abnormal: {(y_train==1).sum()}")
print(f"Test - Normal: {(y_test==0).sum()}, Abnormal: {(y_test==1).sum()}")

# ==========================================
# FEATURE SCALING
# ==========================================

print("\n6. FEATURE SCALING")
print("-" * 80)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_features)
X_test_scaled = scaler.transform(X_test_features)

print(f"Scaled feature range: [{X_train_scaled.min():.2f}, {X_train_scaled.max():.2f}]")

# ==========================================
# MODEL TRAINING WITH COMPUTATIONAL COST TRACKING
# ==========================================

print("\n7. MODEL TRAINING WITH COMPUTATIONAL COST ANALYSIS")
print("-" * 80)

models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'Neural Network': MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)
}

results = {}
computational_costs = {}

for name, model in models.items():
    print(f"\nTraining {name}...")

    # Track computational cost
    start_time = time.time()
    start_memory = get_memory_usage()

    # Train model
    model.fit(X_train_scaled, y_train)

    end_time = time.time()
    end_memory = get_memory_usage()

    # Record costs
    training_time = end_time - start_time
    memory_used = end_memory - start_memory

    computational_costs[name] = {
        'Training Time (s)': training_time,
        'Memory Usage (MB)': memory_used,
        'Peak Memory (MB)': end_memory
    }

    # Predictions
    y_pred = model.predict(X_test_scaled)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]

    # Calculate metrics
    accuracy = model.score(X_test_scaled, y_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    auc = roc_auc_score(y_test, y_proba)
    mcc = matthews_corrcoef(y_test, y_pred)
    kappa = cohen_kappa_score(y_test, y_pred)

    # Clinical metrics
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'precision': report['1']['precision'],
        'recall': report['1']['recall'],
        'sensitivity': sensitivity,  # Same as recall for abnormal class
        'specificity': specificity,
        'f1': report['1']['f1-score'],
        'auc': auc,
        'mcc': mcc,
        'kappa': kappa,
        'y_pred': y_pred,
        'y_proba': y_proba,
        'confusion_matrix': cm,
        'false_negatives': fn,
        'false_positives': fp
    }

    print(f"  Training Time: {training_time:.2f}s")
    print(f"  Memory Used: {memory_used:.2f} MB")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  AUC: {auc:.4f}")
    print(f"  Sensitivity: {sensitivity:.4f}")
    print(f"  Specificity: {specificity:.4f}")
    print(f"  False Negatives: {fn} (Clinical Risk)")

# Save computational cost report
cost_df = pd.DataFrame(computational_costs).T
cost_df.to_csv(f"{RESULTS_DIR}/computational_cost_analysis.csv")
print(f"\n✓ Computational cost analysis saved to {RESULTS_DIR}/computational_cost_analysis.csv")

# ==========================================
# CROSS-VALIDATION
# ==========================================

print("\n8. CROSS-VALIDATION ANALYSIS ")
print("-" * 80)

cv_results = {}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
    print(f"\nCross-validating {name}...")

    # Perform cross-validation
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)

    cv_results[name] = {
        'mean_auc': cv_scores.mean(),
        'std_auc': cv_scores.std(),
        'scores': cv_scores,
        '95%_ci_lower': cv_scores.mean() - 1.96 * (cv_scores.std() / np.sqrt(5)),
        '95%_ci_upper': cv_scores.mean() + 1.96 * (cv_scores.std() / np.sqrt(5))
    }

    print(f"  Mean AUC: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    print(f"  95% CI: [{cv_results[name]['95%_ci_lower']:.4f}, {cv_results[name]['95%_ci_upper']:.4f}]")

# ==========================================
# STATISTICAL SIGNIFICANCE TESTING
# ==========================================

print("\n9. STATISTICAL SIGNIFICANCE TESTING")
print("-" * 80)

# Compare best model (Random Forest) with others
best_model_name = 'Random Forest'
statistical_tests = {}

for name in models.keys():
    if name != best_model_name:
        # McNemar's test for paired predictions
        rf_pred = results[best_model_name]['y_pred']
        other_pred = results[name]['y_pred']

        # Create contingency table
        both_correct = np.sum((rf_pred == y_test) & (other_pred == y_test))
        rf_correct_other_wrong = np.sum((rf_pred == y_test) & (other_pred != y_test))
        rf_wrong_other_correct = np.sum((rf_pred != y_test) & (other_pred == y_test))
        both_wrong = np.sum((rf_pred != y_test) & (other_pred != y_test))

        # McNemar's test
        contingency_table = [[both_correct, rf_correct_other_wrong],
                           [rf_wrong_other_correct, both_wrong]]

        # Calculate McNemar statistic
        b = rf_correct_other_wrong
        c = rf_wrong_other_correct
        mcnemar_stat = ((abs(b - c) - 1)**2) / (b + c) if (b + c) > 0 else 0
        p_value = 1 - stats.chi2.cdf(mcnemar_stat, 1)

        statistical_tests[f"{best_model_name} vs {name}"] = {
            'McNemar Statistic': mcnemar_stat,
            'p-value': p_value,
            'Significant (p<0.05)': p_value < 0.05,
            'RF correct, Other wrong': rf_correct_other_wrong,
            'RF wrong, Other correct': rf_wrong_other_correct
        }

        print(f"\n{best_model_name} vs {name}:")
        print(f"  McNemar χ² = {mcnemar_stat:.4f}, p = {p_value:.4f}")
        print(f"  Significant: {'Yes' if p_value < 0.05 else 'No'} (α=0.05)")

# Save statistical test results
stat_df = pd.DataFrame(statistical_tests).T
stat_df.to_csv(f"{RESULTS_DIR}/statistical_significance_tests.csv")
print(f"\n✓ Statistical tests saved to {RESULTS_DIR}/statistical_significance_tests.csv")

# ==========================================
# INTERPRETABILITY: SHAP VALUES
# ==========================================

print("\n10. MODEL INTERPRETABILITY - SHAP ANALYSIS")
print("-" * 80)

best_model = results[best_model_name]['model']

print(f"Generating SHAP values for {best_model_name}...")
try:
    # Use TreeExplainer for tree-based models
    explainer = shap.TreeExplainer(best_model)
    shap_values = explainer.shap_values(X_test_scaled)

    # For binary classification, get values for class 1 (Abnormal)
    if isinstance(shap_values, list):
        shap_values_class1 = shap_values[1]
    else:
        shap_values_class1 = shap_values

    # Summary plot
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values_class1, X_test_scaled,
                     feature_names=X_combined_features.columns,
                     show=False, max_display=15)
    plt.title(f'SHAP Summary Plot - {best_model_name} (Abnormal Class)', fontsize=14)
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/shap_summary_plot.png", dpi=300, bbox_inches='tight')
    plt.close()

    # Feature importance plot
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values_class1, X_test_scaled,
                     feature_names=X_combined_features.columns,
                     plot_type="bar", show=False, max_display=15)
    plt.title(f'SHAP Feature Importance - {best_model_name}', fontsize=14)
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/shap_feature_importance.png", dpi=300, bbox_inches='tight')
    plt.close()

    print(f"✓ SHAP plots saved to {RESULTS_DIR}/")

    # Get top 10 most important features
    feature_importance = np.abs(shap_values_class1).mean(axis=0)
    feature_names = X_combined_features.columns
    top_indices = np.argsort(feature_importance)[-10:][::-1]

    print("\nTop 10 Most Important Features (SHAP):")
    for idx in top_indices:
        print(f"  {feature_names[idx]}: {feature_importance[idx]:.4f}")

except Exception as e:
    print(f"Error generating SHAP plots: {e}")

# ==========================================
# HYPERPARAMETER TUNING
# ==========================================

print("\n11. HYPERPARAMETER OPTIMIZATION")
print("-" * 80)

# Grid search for Random Forest
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

print("Performing GridSearchCV for Random Forest...")
rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    rf_param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

rf_grid.fit(X_train_scaled, y_train)

print(f"\nBest parameters: {rf_grid.best_params_}")
print(f"Best cross-validation AUC: {rf_grid.best_score_:.4f}")

# Evaluate tuned model
tuned_model = rf_grid.best_estimator_
y_pred_tuned = tuned_model.predict(X_test_scaled)
y_proba_tuned = tuned_model.predict_proba(X_test_scaled)[:, 1]
auc_tuned = roc_auc_score(y_test, y_proba_tuned)

print(f"Tuned model test AUC: {auc_tuned:.4f}")
print(f"Improvement: {auc_tuned - results[best_model_name]['auc']:.4f}")

# ==========================================
# VISUALIZATION AND REPORTING
# ==========================================

print("\n12. GENERATING COMPREHENSIVE REPORTS")
print("-" * 80)

# Model comparison table
metrics_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': [results[m]['accuracy'] for m in results],
    'Precision': [results[m]['precision'] for m in results],
    'Recall/Sensitivity': [results[m]['recall'] for m in results],
    'Specificity': [results[m]['specificity'] for m in results],
    'F1 Score': [results[m]['f1'] for m in results],
    'AUC': [results[m]['auc'] for m in results],
    'MCC': [results[m]['mcc'] for m in results],
    'Kappa': [results[m]['kappa'] for m in results],
    'False Negatives': [results[m]['false_negatives'] for m in results],
    'Training Time (s)': [computational_costs[m]['Training Time (s)'] for m in results]
})

metrics_df.to_csv(f"{RESULTS_DIR}/comprehensive_model_comparison.csv", index=False)
print(f"✓ Model comparison saved to {RESULTS_DIR}/comprehensive_model_comparison.csv")

# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. ROC Curves with CI
ax = axes[0, 0]
for name, result in results.items():
    fpr, tpr, _ = roc_curve(y_test, result['y_proba'])
    ci_lower = cv_results[name]['95%_ci_lower']
    ci_upper = cv_results[name]['95%_ci_upper']
    ax.plot(fpr, tpr, label=f"{name} (AUC={result['auc']:.4f}, 95% CI: [{ci_lower:.3f}, {ci_upper:.3f}])")
ax.plot([0, 1], [0, 1], 'k--')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curves with 95% Confidence Intervals')
ax.legend(loc='lower right', fontsize=8)
ax.grid(True)

# 2. Computational Cost Comparison
ax = axes[0, 1]
models_list = list(computational_costs.keys())
training_times = [computational_costs[m]['Training Time (s)'] for m in models_list]
ax.bar(models_list, training_times, color='steelblue')
ax.set_xlabel('Model')
ax.set_ylabel('Training Time (seconds)')
ax.set_title('Computational Cost - Training Time')
ax.tick_params(axis='x', rotation=45)

# 3. Memory Usage
ax = axes[0, 2]
memory_usage = [computational_costs[m]['Memory Usage (MB)'] for m in models_list]
ax.bar(models_list, memory_usage, color='coral')
ax.set_xlabel('Model')
ax.set_ylabel('Memory Usage (MB)')
ax.set_title('Computational Cost - Memory Usage')
ax.tick_params(axis='x', rotation=45)

# 4. Clinical Metrics (Sensitivity vs Specificity)
ax = axes[1, 0]
sensitivities = [results[m]['sensitivity'] for m in results]
specificities = [results[m]['specificity'] for m in results]
x = np.arange(len(results))
width = 0.35
ax.bar(x - width/2, sensitivities, width, label='Sensitivity', color='green', alpha=0.7)
ax.bar(x + width/2, specificities, width, label='Specificity', color='blue', alpha=0.7)
ax.set_xlabel('Model')
ax.set_ylabel('Score')
ax.set_title('Clinical Metrics: Sensitivity vs Specificity')
ax.set_xticks(x)
ax.set_xticklabels(list(results.keys()), rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3)

# 5. False Negatives Analysis
ax = axes[1, 1]
false_negatives = [results[m]['false_negatives'] for m in results]
colors = ['red' if fn == max(false_negatives) else 'orange' for fn in false_negatives]
ax.bar(list(results.keys()), false_negatives, color=colors)
ax.set_xlabel('Model')
ax.set_ylabel('False Negatives')
ax.set_title('False Negative Analysis (Clinical Risk)')
ax.tick_params(axis='x', rotation=45)
ax.grid(True, alpha=0.3)

# 6. AUC Comparison with Error Bars
ax = axes[1, 2]
models_list = list(results.keys())
aucs = [results[m]['auc'] for m in models_list]
stds = [cv_results[m]['std_auc'] for m in models_list]
ax.bar(models_list, aucs, yerr=stds, capsize=5, color='purple', alpha=0.7)
ax.set_xlabel('Model')
ax.set_ylabel('AUC')
ax.set_title('AUC Comparison with Standard Deviation')
ax.tick_params(axis='x', rotation=45)
ax.set_ylim([0.9, 1.0])
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{RESULTS_DIR}/comprehensive_analysis.png", dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Comprehensive analysis plot saved to {RESULTS_DIR}/comprehensive_analysis.png")

# ==========================================
# FINAL SUMMARY REPORT
# ==========================================

print("\n" + "="*80)
print("FINAL SUMMARY REPORT")
print("="*80)

summary_report = f"""
ECG CLASSIFICATION - COMPREHENSIVE ANALYSIS REPORT
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

DATASET INFORMATION:
- Total Samples: {len(y_combined)}
- Training Samples: {len(y_train)}
- Test Samples: {len(y_test)}
- Features Extracted: {X_combined_features.shape[1]}
- Class Balance (Train): Normal={((y_train==0).sum())}, Abnormal={((y_train==1).sum())}

BEST MODEL: Random Forest
- Test Accuracy: {results['Random Forest']['accuracy']:.4f}
- Test AUC: {results['Random Forest']['auc']:.4f}
- Cross-validation AUC: {cv_results['Random Forest']['mean_auc']:.4f} ± {cv_results['Random Forest']['std_auc']:.4f}
- 95% CI: [{cv_results['Random Forest']['95%_ci_lower']:.4f}, {cv_results['Random Forest']['95%_ci_upper']:.4f}]
- Sensitivity: {results['Random Forest']['sensitivity']:.4f}
- Specificity: {results['Random Forest']['specificity']:.4f}
- False Negatives: {results['Random Forest']['false_negatives']}
- Training Time: {computational_costs['Random Forest']['Training Time (s)']:.2f} seconds
- Memory Usage: {computational_costs['Random Forest']['Memory Usage (MB)']:.2f} MB

STATISTICAL SIGNIFICANCE:
Random Forest vs other models - see statistical_significance_tests.csv for details

INTERPRETABILITY:
- SHAP values generated for feature importance
- Top features identified and visualized

FILES GENERATED:
1. comprehensive_model_comparison.csv
2. computational_cost_analysis.csv
3. statistical_significance_tests.csv
4. dataset_characteristics.txt
5. shap_summary_plot.png
6. shap_feature_importance.png
7. comprehensive_analysis.png

PROBLEMS ADDRESSED:
✓ Q1: Dataset characteristics documented
✓ Q2: Note on patient-level splitting added
✓ Q3: Class imbalance handled via stratification
✓ Q5: Features extracted and analyzed
✓ Q6: Cross-validation and hyperparameter tuning performed
✓ Q7: Computational cost fully analyzed
✓ Q8: Clinical metrics (sensitivity, specificity, FN) reported
✓ Q9: Statistical significance tests performed
✓ Q11: SHAP interpretability analysis completed
"""

with open(f"{RESULTS_DIR}/SUMMARY_REPORT.txt", 'w') as f:
    f.write(summary_report)

print(summary_report)
print(f"\n✓ Full summary report saved to {RESULTS_DIR}/SUMMARY_REPORT.txt")
print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("="*80)


LIME not available. Install with: pip install lime
ENHANCED ECG CLASSIFICATION WITH STATISTICAL ANALYSIS AND INTERPRETABILITY

1. LOADING DATASETS...
--------------------------------------------------------------------------------
✓ MIT-BIH train: (21892, 188)
✓ MIT-BIH test: (21892, 188)
✓ PTB-DB normal: (4046, 188)
✓ PTB-DB abnormal: (10506, 188)

2. DATASET CHARACTERISTICS
--------------------------------------------------------------------------------

MIT-BIH:
  Source: MIT-BIH Arrhythmia Database
  Sampling Rate: 125 Hz (resampled from 360 Hz)
  Signal Length: 187 samples per heartbeat
  Number of Leads: 1 lead (modified lead II)
  Total Samples Train: 21892
  Total Samples Test: 21892

PTB-DB:
  Source: PTB Diagnostic ECG Database
  Sampling Rate: 125 Hz
  Signal Length: 187 samples per heartbeat
  Number of Leads: 1 lead (preprocessed)
  Normal Samples: 4046
  Abnormal Samples: 10506

3. PREPARING BINARY CLASSIFICATION
-----------------------------------------------------------